In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import gc
from tqdm import tqdm

In [3]:
import pandas as pd

def set_table_dtypes(df):
    for col in df.columns:
        if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
            df[col] = df[col].astype('Int64')
        elif col in ["date_decision"]:
            df[col] = pd.to_datetime(df[col])
        elif col[-1] in ("P", "A"):
            df[col] = df[col].astype(float)
        elif col[-1] in ("D",):
            df[col] = pd.to_datetime(df[col])
    return df

def convert_strings(df):
    for col in df.columns:
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            df[col] = df[col].astype(pd.CategoricalDtype(categories=new_categories, ordered=True))
    return df


# Test base

In [4]:
test_base = pd.read_parquet('/kaggle/input/home-credit-credit-risk-modeling/test.parquet')
test_base = set_table_dtypes(test_base)
test_base.drop(columns=['date_decision','MONTH'], inplace=True)
test_base

,case_id
0,14256
1,1348
2,13475
3,1120
4,11878
...,...
19995,3444
19996,19016
19997,4171
19998,10063


# Part Static

In [5]:
test_static = pd.read_parquet('/kaggle/input/home-credit-credit-risk-modeling/test_dataset/transformed/test_static_0_0.parquet')
test_static = set_table_dtypes(test_static)
test_static

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,9769,NaN,NaN,2350.4001,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,NaN,0.00,0.00,NaN,BO,AL,NaT
1,9935,0.0,9912.200,4079.0000,0.0000,0.0,2.0,0.0,0.0,0.0,...,1.0,2.0,81625.6000,81625.6000,81625.60,9947.00,7017.4,FO,AL,NaT
2,6596,NaN,NaN,8287.6010,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,NaN,0.00,0.00,NaN,BO,None,NaT
3,4560,NaN,NaN,4750.8003,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,NaN,NaN,0.00,0.00,NaN,BO,AL,NaT
4,7483,NaN,20071.123,2749.4001,0.0000,0.0,1.0,8.0,0.0,0.0,...,0.0,4.0,NaN,NaN,0.00,0.00,NaN,BO,None,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17771,18237,0.0,21564.600,4859.6000,1807.6000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1787.6000,1787.6000,1787.60,54739.80,1809.0,FO,None,NaT
17772,16737,0.0,26411.201,6714.0000,1028.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,2005.2001,2005.2001,1005.20,111427.60,6087.0,FO,None,NaT
17773,16074,0.0,271682.000,6112.2000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,4.0,0.0000,0.0000,0.00,438058.00,NaN,BO,None,NaT
17774,16103,0.0,39812.800,6216.8003,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0000,0.0000,0.00,39820.20,NaN,FO,None,NaT


In [6]:
feature_list = ['case_id','actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'daysoverduetolerancedd_3976961L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'eir_270L', 'homephncnt_628L', 'inittransactionamount_650A', 'interestrate_311L', 'interestrategrace_34L', 'lastapprcredamount_781A', 'lastdependentsnum_448L', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcredamount_222A', 'maininc_215A', 'mastercontrelectronic_519L', 'mastercontrexist_109L', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdbddpdlast1m_3658939P', 'maxdbddpdtollast12m_3658940P', 'maxdbddpdtollast6m_4187119P', 'maxdebt4_972A', 'maxdpdfrom6mto36m_3546853P', 'maxdpdinstlnum_3546846P', 'maxdpdlast12m_727P', 'maxdpdlast24m_143P', 'maxdpdlast3m_392P', 'maxdpdlast6m_474P', 'maxdpdlast9m_1059P', 'maxdpdtolerance_374P', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'mindbddpdlast24m_3658935P', 'mindbdtollast24m_4525191P', 'mobilephncnt_593L', 'monthsannuity_845L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numincomingpmts_3546848L', 'numinstlallpaidearly3d_817L', 'numinstls_657L', 'numinstlsallpaid_934L', 'numinstlswithdpd10_728L', 'numinstlswithdpd5_4187116L', 'numinstlswithoutdpd_562L', 'numinstmatpaidtearly2d_4499204L', 'numinstpaid_4499208L', 'numinstpaidearly3d_3546850L', 'numinstpaidearly3dest_4493216L', 'numinstpaidearly5d_1087L', 'numinstpaidearly5dest_4493211L', 'numinstpaidearly5dobd_4499205L', 'numinstpaidearly_338L', 'numinstpaidearlyest_4493214L', 'numinstpaidlastcontr_4325080L', 'numinstpaidlate1d_3546852L', 'numinstregularpaid_973L', 'numinstregularpaidest_4493210L', 'numinsttopaygr_769L', 'numinsttopaygrest_4493213L', 'numinstunpaidmax_3546851L', 'numinstunpaidmaxest_4493212L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'pctinstlsallpaidearl3d_427L', 'pctinstlsallpaidlat10d_839L', 'pctinstlsallpaidlate1d_3546856L', 'pctinstlsallpaidlate4d_3546849L', 'pctinstlsallpaidlate6d_3546844L', 'pmtnum_254L', 'posfpd10lastmonth_333P', 'posfpd30lastmonth_3976960P', 'posfstqpd30lastmonth_3976962P', 'price_1097A', 'sellerplacecnt_915L', 'sellerplacescnt_216L', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallast1m_4525188A', 'actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L', 'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A', 'credacc_maxhisbal_375A', 'credacc_minhisbal_90A', 'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A', 'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A', 'tenor_203L', 'contractssum_5085716L', 'days120_123L', 'days180_256L', 'days30_165L', 'days360_512L', 'days90_310L', 'firstquarter_103L', 'for3years_128L', 'for3years_504L', 'for3years_584L', 'formonth_118L', 'formonth_206L', 'formonth_535L', 'forquarter_1017L', 'forquarter_462L', 'forquarter_634L', 'fortoday_1092L', 'forweek_1077L', 'forweek_528L', 'forweek_601L', 'foryear_618L', 'foryear_818L', 'foryear_850L', 'fourthquarter_440L', 'numberofqueries_373L', 'pmtaverage_3A', 'pmtaverage_4527227A', 'pmtaverage_4955615A', 'pmtcount_4527229L', 'pmtcount_4955617L', 'pmtcount_693L', 'pmtscount_423L', 'pmtssum_45A', 'secondquarter_766L', 'thirdquarter_1082L', 'amount_4527230A', 'amount_4917619A', 'amount_416A', 'actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_136L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'daysoverduetolerancedd_3976961L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'downpmt_116A', 'eir_270L', 'homephncnt_628L', 'inittransactionamount_650A', 'interestrate_311L', 'interestrategrace_34L', 'lastapprcredamount_781A', 'lastdependentsnum_448L', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcredamount_222A', 'maininc_215A', 'mastercontrelectronic_519L', 'mastercontrexist_109L', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdbddpdlast1m_3658939P', 'maxdbddpdtollast12m_3658940P', 'maxdbddpdtollast6m_4187119P', 'maxdebt4_972A', 'maxdpdfrom6mto36m_3546853P', 'maxdpdinstlnum_3546846P', 'maxdpdlast12m_727P', 'maxdpdlast24m_143P', 'maxdpdlast3m_392P', 'maxdpdlast6m_474P', 'maxdpdlast9m_1059P', 'maxdpdtolerance_374P', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'mindbddpdlast24m_3658935P', 'mindbdtollast24m_4525191P', 'mobilephncnt_593L', 'monthsannuity_845L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numincomingpmts_3546848L', 'numinstlallpaidearly3d_817L', 'numinstls_657L', 'numinstlsallpaid_934L', 'numinstlswithdpd10_728L', 'numinstlswithdpd5_4187116L', 'numinstlswithoutdpd_562L', 'numinstmatpaidtearly2d_4499204L', 'numinstpaid_4499208L', 'numinstpaidearly3d_3546850L', 'numinstpaidearly3dest_4493216L', 'numinstpaidearly5d_1087L', 'numinstpaidearly5dest_4493211L', 'numinstpaidearly5dobd_4499205L', 'numinstpaidearly_338L', 'numinstpaidearlyest_4493214L', 'numinstpaidlastcontr_4325080L', 'numinstpaidlate1d_3546852L', 'numinstregularpaid_973L', 'numinstregularpaidest_4493210L', 'numinsttopaygr_769L', 'numinsttopaygrest_4493213L', 'numinstunpaidmax_3546851L', 'numinstunpaidmaxest_4493212L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'pctinstlsallpaidearl3d_427L', 'pctinstlsallpaidlat10d_839L', 'pctinstlsallpaidlate1d_3546856L', 'pctinstlsallpaidlate4d_3546849L', 'pctinstlsallpaidlate6d_3546844L', 'pmtnum_254L', 'posfpd10lastmonth_333P', 'posfpd30lastmonth_3976960P', 'posfstqpd30lastmonth_3976962P', 'price_1097A', 'sellerplacecnt_915L', 'sellerplacescnt_216L', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallast1m_4525188A', 'actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L', 'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A', 'credacc_maxhisbal_375A', 'credacc_minhisbal_90A', 'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A', 'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A', 'tenor_203L']

In [32]:
new_feature_list = ['case_id', 'employername_160M', 'num_group1', 'pmtamount_36A', 'processingdate_168D', 'actualdpdtolerance_344P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_629L', 'applicationscnt_867L', 'avgdbddpdlast24m_3658932P', 'avgdbddpdlast3m_4187120P', 'avgdbdtollast24m_4525197P', 'avgdpdtolclosure24_3658938P', 'avginstallast24m_3658937A', 'avgmaxdpdlast9m_3716943P', 'clientscnt12m_3712952L', 'clientscnt3m_3712950L', 'clientscnt6m_3712949L', 'clientscnt_100L', 'clientscnt_1022L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'credamount_770A', 'credtype_322L', 'currdebt_22A', 'currdebtcredtyperange_828A', 'daysoverduetolerancedd_3976961L', 'deferredmnthsnum_166L', 'disbursedcredamount_1113A', 'disbursementtype_67L', 'downpmt_116A', 'eir_270L', 'firstdatedue_489D', 'homephncnt_628L', 'inittransactioncode_186L', 'interestrate_311L', 'isbidproduct_1095L', 'lastactivateddate_801D', 'lastapplicationdate_877D', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastapprdate_640D', 'lastcancelreason_561M', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'lastst_736L', 'maininc_215A', 'mastercontrelectronic_519L', 'mastercontrexist_109L', 'maxannuity_159A', 'maxdbddpdlast1m_3658939P', 'maxdbddpdtollast12m_3658940P', 'maxdbddpdtollast6m_4187119P', 'maxdebt4_972A', 'maxdpdfrom6mto36m_3546853P', 'maxdpdinstlnum_3546846P', 'maxdpdlast12m_727P', 'maxdpdlast24m_143P', 'maxdpdlast3m_392P', 'maxdpdlast6m_474P', 'maxdpdlast9m_1059P', 'maxdpdtolerance_374P', 'maxinstallast24m_3658928A', 'mindbddpdlast24m_3658935P', 'mindbdtollast24m_4525191P', 'mobilephncnt_593L', 'monthsannuity_845L', 'numactivecreds_622L', 'numactivecredschannel_414L', 'numactiverelcontr_750L', 'numcontrs3months_479L', 'numincomingpmts_3546848L', 'numinstlallpaidearly3d_817L', 'numinstls_657L', 'numinstlsallpaid_934L', 'numinstlswithdpd10_728L', 'numinstlswithdpd5_4187116L', 'numinstlswithoutdpd_562L', 'numinstpaidearly3d_3546850L', 'numinstpaidearly5d_1087L', 'numinstpaidearly_338L', 'numinstpaidlate1d_3546852L', 'numinstregularpaid_973L', 'numinsttopaygr_769L', 'numinstunpaidmax_3546851L', 'numnotactivated_1143L', 'numpmtchanneldd_318L', 'numrejects9m_859L', 'opencred_647L', 'paytype1st_925L', 'paytype_783L', 'pctinstlsallpaidearl3d_427L', 'pctinstlsallpaidlat10d_839L', 'pctinstlsallpaidlate1d_3546856L', 'pctinstlsallpaidlate4d_3546849L', 'pctinstlsallpaidlate6d_3546844L', 'pmtnum_254L', 'posfpd10lastmonth_333P', 'posfpd30lastmonth_3976960P', 'posfstqpd30lastmonth_3976962P', 'previouscontdistrict_112M', 'price_1097A', 'sellerplacecnt_915L', 'sellerplacescnt_216L', 'sumoutstandtotal_3546847A', 'totaldebt_9A', 'totalsettled_863A', 'twobodfilling_608L', 'actualdpd_943P', 'annuity_853A', 'approvaldate_319D', 'cancelreason_3545846M', 'creationdate_885D', 'credacc_credlmt_575A', 'credamount_590A', 'credtype_587L', 'currdebt_94A', 'dateactivated_425D', 'district_544M', 'downpmt_134A', 'education_1138M', 'familystate_726L', 'firstnonzeroinstldate_307D', 'inittransactioncode_279L', 'isbidproduct_390L', 'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'outstandingdebt_522A', 'pmtnum_8L', 'postype_4733339M', 'profession_152M', 'rejectreason_755M', 'rejectreasonclient_4145042M', 'status_219L', 'tenor_203L', 'dateofbirth_337D', 'days120_123L', 'days180_256L', 'days30_165L', 'days360_512L', 'days90_310L', 'description_5085714M', 'education_1103M', 'education_88M', 'firstquarter_103L', 'fourthquarter_440L', 'maritalst_385M', 'maritalst_893M', 'numberofqueries_373L', 'secondquarter_766L', 'thirdquarter_1082L', 'amount_4527230A', 'name_4527232M', 'recorddate_4527225D', 'amount_4917619A', 'deductiondate_4917603D', 'name_4917606M', 'amount_416A', 'openingdate_313D','pmtssum_45A']

In [7]:
colll = test_static.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        test_static.drop(columns=c, inplace=True)
test_static.head()

100%|██████████| 168/168 [00:00<00:00, 356.20it/s]


,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
0,9769,NaN,NaN,2350.4001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,1.0,NaN,NaN,0.0,0.0,NaN
1,9935,0.0,9912.200,4079.0000,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,NaN,1.0,2.0,81625.6,81625.6,81625.6,9947.0,7017.4
2,6596,NaN,NaN,8287.6010,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,NaN
3,4560,NaN,NaN,4750.8003,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,NaN
4,7483,NaN,20071.123,2749.4001,0.0,0.0,1.0,8.0,0.0,0.0,...,0.0,0.0,NaN,0.0,4.0,NaN,NaN,0.0,0.0,NaN


In [8]:
merged_df = pd.merge(test_base, test_static, on='case_id', how='left')
merged_df

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A
0,14256,0.0,261076.58,7214.8003,8845.8000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,6.0,107464.960,107464.96,107464.96,719418.600,17691.6
1,1348,0.0,43920.20,1113.2001,0.0000,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,21204.800,0.00,0.00,56542.000,NaN
2,13475,0.0,348415.47,1746.0000,1059.0000,0.0,10.0,0.0,0.0,0.0,...,0.0,0.0,NaN,3.0,3.0,107443.805,0.00,183676.53,348148.030,7244.4
3,1120,0.0,68781.40,9718.2000,1726.6000,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,50400.0,2.0,6.0,90606.600,90606.60,90606.60,104708.195,1726.6
4,11878,0.0,52085.20,1788.0000,5960.8003,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,31551.8,1.0,5.0,46755.370,46755.37,46755.37,917269.200,5141.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3444,0.0,15261.80,1343.6000,5272.8003,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,11538.0,0.0,4.0,58694.000,58694.00,58694.00,0.000,8547.8
19996,19016,0.0,25281.00,9427.2000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,0.0,1.0,0.000,0.00,140036.36,68363.000,NaN
19997,4171,0.0,40704.00,4530.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,74000.0,1.0,8.0,0.000,0.00,0.00,76639.086,NaN
19998,10063,0.0,0.00,1136.8000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10998.0,0.0,2.0,0.000,0.00,0.00,35510.402,NaN


In [9]:
test_static_cb = pd.read_parquet('/kaggle/input/home-credit-credit-risk-modeling/test_dataset/transformed/test_static_cb_0.parquet')
test_static_cb = set_table_dtypes(test_static_cb)
test_static_cb

,case_id,assignmentdate_238D,assignmentdate_4527235D,assignmentdate_4955616D,birthdate_574D,contractssum_5085716L,dateofbirth_337D,dateofbirth_342D,days120_123L,days180_256L,...,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
0,9935,NaT,NaT,NaT,NaT,NaN,1992-09-12,NaT,2.0,4.0,...,NaN,NaN,DEDUCTION_6,NaT,2020-12-04,NaT,None,NaN,3.0,15.0
1,6596,NaT,NaT,NaT,NaT,NaN,1985-09-13,NaT,6.0,0.0,...,NaN,NaN,None,NaT,2020-12-04,NaT,None,NaN,10.0,1.0
2,4560,NaT,2020-12-05,NaT,NaT,NaN,1961-03-13,NaT,5.0,5.0,...,NaN,NaN,PENSION_6,NaT,NaT,NaT,None,NaN,1.0,3.0
3,7483,NaT,NaT,NaT,NaT,NaN,1996-09-12,NaT,0.0,4.0,...,NaN,NaN,DEDUCTION_6,NaT,2020-12-05,NaT,None,NaN,0.0,2.0
4,7462,NaT,NaT,NaT,NaT,NaN,1995-01-11,NaT,0.0,0.0,...,NaN,NaN,DEDUCTION_6,NaT,2020-12-04,NaT,None,NaN,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16195,16737,NaT,NaT,2010-05-13,NaT,481465.42,1952-04-12,1952-04-12,2.0,3.0,...,NaN,NaN,None,NaT,NaT,2021-05-20,6% - 8%,0.111043,4.0,4.0
16196,16074,NaT,NaT,NaT,NaT,281583.86,1961-04-13,NaT,0.0,4.0,...,NaN,NaN,None,NaT,NaT,2021-05-20,None,NaN,1.0,2.0
16197,16103,NaT,NaT,NaT,NaT,2570475.44,1974-01-11,NaT,2.0,3.0,...,NaN,NaN,None,NaT,NaT,2021-05-15,None,NaN,6.0,8.0
16198,10750,NaT,NaT,2013-11-25,NaT,305795.31,1955-11-11,NaT,0.0,0.0,...,NaN,NaN,None,NaT,NaT,2021-05-20,None,NaN,5.0,1.0


In [10]:
colll = test_static_cb.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        test_static_cb.drop(columns=c, inplace=True)
test_static_cb.head()

100%|██████████| 53/53 [00:00<00:00, 1097.78it/s]


,case_id,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,...,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,secondquarter_766L,thirdquarter_1082L
0,9935,NaN,2.0,4.0,0.0,17.0,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,15.0
1,6596,NaN,6.0,0.0,3.0,17.0,3.0,10.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0
2,4560,NaN,5.0,5.0,2.0,5.0,3.0,8.0,NaN,NaN,...,NaN,3020.6,NaN,13.0,NaN,NaN,NaN,NaN,1.0,3.0
3,7483,NaN,0.0,4.0,0.0,5.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0
4,7462,NaN,0.0,0.0,0.0,2.0,0.0,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,2.0


In [11]:
merged_df = pd.merge(merged_df, test_static_cb, on='case_id', how='left')
merged_df

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,secondquarter_766L,thirdquarter_1082L
0,14256,0.0,261076.58,7214.8003,8845.8000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,32931.6,NaN,14.0,NaN,NaN,NaN,5.0,6.0
1,1348,0.0,43920.20,1113.2001,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,13475,0.0,348415.47,1746.0000,1059.0000,0.0,10.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1120,0.0,68781.40,9718.2000,1726.6000,0.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,7.0
4,11878,0.0,52085.20,1788.0000,5960.8003,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3444,0.0,15261.80,1343.6000,5272.8003,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0
19996,19016,0.0,25281.00,9427.2000,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
19997,4171,0.0,40704.00,4530.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0
19998,10063,0.0,0.00,1136.8000,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0


In [38]:
len(set(merged_df['case_id']))

20000

# Part app

In [12]:
dataPath = '/kaggle/input/home-credit-credit-risk-modeling/test_dataset/transformed'
applprev_path = [
    dataPath + "/test_applprev_1_0.parquet",
] 
test_applprev = pd.concat(
    [pd.read_parquet(file_path).pipe(set_table_dtypes) for file_path in applprev_path],
    axis=0,  # Concatenate along the rows (vertically)
    ignore_index=True  # Reset index after concatenation
)
test_applprev = test_applprev[test_applprev['case_id'] < 1526660]
test_applprev

,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,...,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,9769,0.0,2685.2000,NaT,NaN,P11_156_146,NaN,2018-04-30,NaN,0.0,...,1,NaN,12.0,P177_117_192,a55475b1,a55475b1,a55475b1,NaN,T,12.0
1,9769,0.0,4618.4000,NaT,NaN,P94_109_143,NaN,2018-06-04,NaN,0.0,...,0,NaN,12.0,P46_145_78,a55475b1,P94_109_143,P94_109_143,NaN,D,12.0
2,9769,0.0,5995.2000,NaT,NaN,P94_109_143,NaN,2017-07-26,NaN,0.0,...,3,NaN,6.0,P177_117_192,a55475b1,P99_56_166,a55475b1,NaN,D,6.0
3,9769,0.0,15462.2000,NaT,NaN,a55475b1,NaN,2017-07-28,NaN,0.0,...,2,NaN,24.0,P177_117_192,a55475b1,P94_109_143,P94_109_143,NaN,D,24.0
4,9935,0.0,0.0000,2020-09-28,NaN,a55475b1,NaN,2020-09-28,NaN,37498.0,...,2,20870.00,18.0,P177_117_192,a55475b1,a55475b1,a55475b1,800005000.0,A,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99572,17499,0.0,1920.0000,2019-12-22,NaN,a55475b1,NaN,2019-12-22,NaN,0.0,...,4,0.00,12.0,P149_40_170,a55475b1,a55475b1,a55475b1,NaN,K,12.0
99573,17499,0.0,2098.4001,2019-10-21,NaN,a55475b1,NaN,2019-10-21,0.0,0.0,...,5,0.00,12.0,P149_40_170,a55475b1,a55475b1,a55475b1,NaN,K,12.0
99574,17499,0.0,3069.8000,2020-11-15,NaN,a55475b1,NaN,2020-11-15,NaN,0.0,...,1,18241.40,12.0,P46_145_78,a55475b1,a55475b1,a55475b1,NaN,A,12.0
99575,17499,0.0,4933.4000,2020-04-18,NaN,a55475b1,NaN,2020-04-18,NaN,0.0,...,3,113331.98,36.0,P149_40_170,a55475b1,a55475b1,a55475b1,NaN,A,36.0


In [13]:
colll = test_applprev.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        test_applprev.drop(columns=c, inplace=True)
test_applprev.head()

100%|██████████| 41/41 [00:00<00:00, 138.16it/s]


,case_id,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
0,9769,0.0,2685.2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,25998.0,NaN,0.0,36000.0,NaN,NaN,12.0,NaN,12.0
1,9769,0.0,4618.4,NaN,NaN,NaN,0.0,NaN,NaN,NaN,44000.0,NaN,0.0,8000.0,NaN,NaN,12.0,NaN,12.0
2,9769,0.0,5995.2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,31860.0,NaN,0.0,30000.0,NaN,NaN,6.0,NaN,6.0
3,9769,0.0,15462.2,NaN,NaN,NaN,0.0,NaN,NaN,NaN,10728.0,NaN,8594.0,50000.0,NaN,NaN,24.0,NaN,24.0
4,9935,0.0,0.0,NaN,NaN,NaN,37498.0,NaN,NaN,NaN,37498.0,20870.0,0.0,54000.0,0.0,20870.0,18.0,800005000.0,18.0


In [41]:
len(set(merged_df['case_id']))

20000

In [14]:
test_applprev.columns

Index(['case_id', 'actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L',
       'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A',
       'credacc_maxhisbal_375A', 'credacc_minhisbal_90A',
       'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A',
       'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P',
       'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A',
       'tenor_203L'],
      dtype='object')

In [15]:
columns_to_mean = ['actualdpd_943P', 'annuity_853A', 'byoccupationinc_3656910L',
       'childnum_21L', 'credacc_actualbalance_314A', 'credacc_credlmt_575A',
       'credacc_maxhisbal_375A', 'credacc_minhisbal_90A',
       'credacc_transactions_402L', 'credamount_590A', 'currdebt_94A',
       'downpmt_134A', 'mainoccupationinc_437A', 'maxdpdtolerance_577P',
       'outstandingdebt_522A', 'pmtnum_8L', 'revolvingaccount_394A',
       'tenor_203L']

test_applprev = test_applprev.groupby('case_id')[columns_to_mean].mean()
test_applprev

,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
case_id,,,,,,,,,,,,,,,,,,
1,0.0,6445.000000,NaN,3.00,NaN,0.0,NaN,NaN,NaN,63454.000000,NaN,0.000000,25000.000000,1.000000,0.000000,12.000000,NaN,12.000000
2,0.0,1911.933350,NaN,0.00,45400.0,20270.0,-15410.00,-15410.0,0.0,23328.000000,5989.100167,650.000000,69333.333333,0.166667,7186.920200,8.166667,780664600.0,9.666667
4,0.0,2076.700028,33563.25,0.25,NaN,0.0,NaN,0.0,NaN,29450.333333,1780.074882,222.222222,23267.894737,0.000000,1944.670588,18.944444,NaN,18.764706
5,0.0,9132.500500,1.00,3.00,NaN,0.0,NaN,NaN,NaN,54774.500000,0.000000,10125.500000,50000.000000,NaN,0.000000,6.000000,NaN,6.000000
6,0.0,5993.600000,NaN,NaN,NaN,0.0,NaN,NaN,NaN,59980.000000,0.000000,0.000000,40000.000000,0.000000,11559.601000,12.000000,NaN,12.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.0,3134.500050,NaN,NaN,NaN,0.0,NaN,NaN,NaN,20000.000000,NaN,0.000000,45333.333333,NaN,2111.600000,9.000000,NaN,9.000000
19996,0.0,4752.885729,NaN,0.00,NaN,0.0,NaN,NaN,NaN,44579.771429,0.000000,0.000000,41285.714286,0.000000,0.000000,14.857143,NaN,11.428571
19997,0.0,15646.601000,1.00,2.00,NaN,0.0,NaN,NaN,NaN,44994.000000,0.000000,0.000000,80000.000000,0.000000,0.000000,3.000000,NaN,3.000000


In [16]:
merged_df = pd.merge(merged_df, test_applprev, on='case_id', how='left')
merged_df

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L
0,14256,0.0,261076.58,7214.8003,8845.8000,0.0,0.0,0.0,0.0,0.0,...,0.0,39667.014429,8251.832400,0.000000,66200.000000,4.333333,10746.497200,23.454545,800100300.0,18.666667
1,1348,0.0,43920.20,1113.2001,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,13250.000000,0.000000,0.000000,39600.000000,1.666667,0.000000,9.250000,NaN,6.000000
2,13475,0.0,348415.47,1746.0000,1059.0000,0.0,10.0,0.0,0.0,0.0,...,0.0,49287.670000,19475.871667,0.000000,25983.333333,1.666667,36462.112571,25.882353,780140895.0,27.555556
3,1120,0.0,68781.40,9718.2000,1726.6000,0.0,1.0,0.0,0.0,0.0,...,0.0,37383.166667,22651.650000,0.000000,33666.666667,8.250000,22651.650000,14.500000,800167360.0,14.166667
4,11878,0.0,52085.20,1788.0000,5960.8003,0.0,0.0,0.0,0.0,0.0,...,0.0,25225.975000,5875.285714,500.000000,45428.571429,0.166667,804.052571,8.500000,800267000.0,8.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3444,0.0,15261.80,1343.6000,5272.8003,0.0,0.0,0.0,0.0,0.0,...,NaN,42300.033333,9782.333500,0.000000,40000.000000,0.666667,11738.800200,17.000000,NaN,16.333333
19996,19016,0.0,25281.00,9427.2000,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,40550.000000,0.000000,0.000000,45750.000000,0.000000,0.000000,14.666667,NaN,14.666667
19997,4171,0.0,40704.00,4530.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,46166.293467,0.000000,64.253335,39249.333333,3.000000,3124.750000,16.928571,729099200.0,23.857143
19998,10063,0.0,0.00,1136.8000,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
len(set(merged_df['case_id']))

20000

# Part Tax

In [17]:
dataPath = '/kaggle/input/home-credit-credit-risk-modeling/test_dataset/transformed'
tax_path = [
    dataPath + "/test_tax_registry_a_1.parquet",
    dataPath + "/test_tax_registry_b_1.parquet",
] 
test_tax = pd.concat(
    [pd.read_parquet(file_path).pipe(set_table_dtypes) for file_path in tax_path],
    axis=0,  # Concatenate along the rows (vertically)
    ignore_index=True  # Reset index after concatenation
)
test_tax = test_tax[test_tax['case_id'] < 1526660]
test_tax

,case_id,amount_4527230A,name_4527232M,num_group1,recorddate_4527225D,amount_4917619A,deductiondate_4917603D,name_4917606M
0,9769,850.0000,8c9460bf,2,2020-12-09,NaN,NaT,NaN
1,9769,3581.6000,8c9460bf,1,2020-12-04,NaN,NaT,NaN
2,9769,3660.2000,8c9460bf,0,2020-12-08,NaN,NaT,NaN
3,9769,4429.8003,8c9460bf,3,2020-12-04,NaN,NaT,NaN
4,9935,428.4000,c75d2f47,3,2020-12-04,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...
81020,17499,NaN,NaN,2,NaT,15254.0,2021-01-14,cda1fd10
81021,17499,NaN,NaN,3,NaT,15254.0,2021-02-18,cda1fd10
81022,17499,NaN,NaN,5,NaT,15254.0,2021-04-16,cda1fd10
81023,17499,NaN,NaN,6,NaT,15254.0,2021-05-15,cda1fd10


In [18]:
colll = test_tax.columns.tolist()
for c in tqdm(colll):
    if c not in feature_list:
        test_tax.drop(columns=c, inplace=True)
test_tax.head()

100%|██████████| 8/8 [00:00<00:00, 524.94it/s]


,case_id,amount_4527230A,amount_4917619A
0,9769,850.0000,NaN
1,9769,3581.6000,NaN
2,9769,3660.2000,NaN
3,9769,4429.8003,NaN
4,9935,428.4000,NaN


In [19]:
columns_to_mean = ['amount_4527230A','amount_4917619A']

test_tax = test_tax.groupby('case_id')[columns_to_mean].mean()
test_tax

,amount_4527230A,amount_4917619A
case_id,,
0,6000.000000,NaN
2,2778.771429,NaN
3,850.000000,NaN
6,4000.000000,NaN
8,1882.857171,NaN
...,...,...
19994,NaN,18334.975000
19995,NaN,6354.533333
19997,NaN,41925.300333


In [20]:
merged_df = pd.merge(merged_df, test_tax, on='case_id', how='left')
merged_df

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L,amount_4527230A,amount_4917619A
0,14256,0.0,261076.58,7214.8003,8845.8000,0.0,0.0,0.0,0.0,0.0,...,8251.832400,0.000000,66200.000000,4.333333,10746.497200,23.454545,800100300.0,18.666667,NaN,NaN
1,1348,0.0,43920.20,1113.2001,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,39600.000000,1.666667,0.000000,9.250000,NaN,6.000000,968.800000,NaN
2,13475,0.0,348415.47,1746.0000,1059.0000,0.0,10.0,0.0,0.0,0.0,...,19475.871667,0.000000,25983.333333,1.666667,36462.112571,25.882353,780140895.0,27.555556,NaN,NaN
3,1120,0.0,68781.40,9718.2000,1726.6000,0.0,1.0,0.0,0.0,0.0,...,22651.650000,0.000000,33666.666667,8.250000,22651.650000,14.500000,800167360.0,14.166667,4476.600100,34885.633333
4,11878,0.0,52085.20,1788.0000,5960.8003,0.0,0.0,0.0,0.0,0.0,...,5875.285714,500.000000,45428.571429,0.166667,804.052571,8.500000,800267000.0,8.500000,NaN,16080.371571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3444,0.0,15261.80,1343.6000,5272.8003,0.0,0.0,0.0,0.0,0.0,...,9782.333500,0.000000,40000.000000,0.666667,11738.800200,17.000000,NaN,16.333333,1649.420699,NaN
19996,19016,0.0,25281.00,9427.2000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,45750.000000,0.000000,0.000000,14.666667,NaN,14.666667,NaN,12175.028714
19997,4171,0.0,40704.00,4530.0000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.000000,64.253335,39249.333333,3.000000,3124.750000,16.928571,729099200.0,23.857143,2115.500083,NaN
19998,10063,0.0,0.00,1136.8000,0.0000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,572.450000


In [50]:
len(set(merged_df['case_id']))

20000

# Done

In [21]:
merged_df.to_parquet('/kaggle/working/test_fea_v4.parquet')